In [1]:
# univariate multi-step lstm
import pandas as pd
import numpy as np
from keras import backend as K
import scipy.stats as st
import kerastuner as kt
import tensorflow as tf
import glob
import json
from collections import defaultdict
from pandas import read_csv
from kerastuner.tuners import Hyperband
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import (LSTM, GRU, Dense, Flatten, RepeatVector, TimeDistributed, )
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.python.keras.callbacks import ModelCheckpoint


In [2]:
df = read_csv("predicted_temperature_and_test_values.csv").dropna()

In [3]:
df

,pred_temperature,temperature,humidity
0,23.652550,23.914773,21.051136
1,23.603794,23.983333,21.000000
2,23.683710,23.966480,20.882682
3,23.418789,23.983146,20.617978
4,23.743364,24.000000,21.406780
...,...,...,...
149,23.686821,24.000000,20.410112
150,23.405550,24.000000,20.016949
151,23.707924,24.000000,20.056180
152,23.755102,24.000000,20.067039


In [4]:
def calculate_confidence(data):
    diff_actual_pred = data.temperature - data.pred_temperature
    confidence_interval= st.t.interval(0.99, diff_actual_pred.shape[0]-1, loc=diff_actual_pred.mean(), scale=st.sem(diff_actual_pred.values.tolist()))
    return confidence_interval

In [5]:
interval = calculate_confidence(df)
print(interval, (interval[1] - interval[0])/2)

(0.12069348286533801, 0.3731482927810138) 0.1262274049578379


In [37]:
n_input=5
dataset = read_csv("predicted_temperature_and_test_values.csv")
# convert history into inputs and outputs
def to_supervised(dataset, n_input):
    # labeling datapoints
    diff_actual_pred = dataset.temperature - dataset.pred_temperature
    interval = calculate_confidence(dataset)
    dataset['label'] = ((diff_actual_pred>interval[0]) & (diff_actual_pred<interval[1]))
    data = dataset.values
    
    X, labels = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        # ensure we have enough data for this instance
        if in_end <= len(data):
            x_input = data[in_start:in_end, 1]
            x_input[-1] = data[in_end-1, 0]
            x_input = x_input.reshape((len(x_input), 1))
            output = data[in_start:in_end, 3].sum()>n_input/2
            X.append(x_input)
            labels.append(output)
        # move along one time step
        in_start += 1
    return np.array(X).astype('float32').reshape((-1,1,n_input)), np.array(labels).astype('float32')

In [7]:
# split a univariate dataset into train/test sets
def split_dataset(data):
    split_index = int((len(data)*0.8))
    train, test = data[:split_index], data[split_index:]
    return train, test

In [13]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# train the model
def build_cnn_model(hp):
    # define hyperparameters parameters
    num_layers = hp.Int('num_layers', 1, 7, step=2)
    activation = hp.Choice("activation", ["sigmoid", "relu", "tanh"])
    hiddent_units = hp.Choice('hiddent_units', [50, 100, 150, 200])

    n_features, n_outputs = 2, 2
    n_timesteps = 1

    # define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='sigmoid', input_shape=(n_timesteps, n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(RepeatVector(n_features))
    # we want sigmoid in the first lstm layer so that we convert the data between 0 and 1
    model.add(LSTM(hiddent_units, return_sequences=num_layers > 1))
    for i in range(num_layers - 2):
        model.add(LSTM(hiddent_units, activation=activation, return_sequences=True, dropout=hp.Float("dropout", 0.0 , 0.8 , 0.2, default=0.2)))
    if num_layers > 1:
        model.add(LSTM(hiddent_units, activation=activation))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', metrics=['accuracy', recall_m, precision_m, f1_score], optimizer='adam')
    return model

def build_lstm_model(hp):
    # define hyperparameters parameters
    num_layers = hp.Int('num_layers', 1, 7, step=2)
    activation = hp.Choice("activation", ["sigmoid", "relu", "tanh"])
    hiddent_units = hp.Choice('hiddent_units', [50, 100, 150, 200])
    n_features, n_outputs = 2, 2
    n_timesteps = 1

    # define model
    model = Sequential()
    # we want sigmoid in the first layer so that we convert the data between 0 and 1
    model.add(LSTM(hiddent_units, activation='sigmoid', input_shape=(n_timesteps, n_features), return_sequences=num_layers > 1))
    for i in range(num_layers - 2):
        model.add(LSTM(hiddent_units, activation=activation, return_sequences=True, dropout=hp.Float("dropout", 0.0 , 0.8 , 0.2)))
    if num_layers > 1:
        model.add(LSTM(hiddent_units, activation=activation))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', metrics=['accuracy', recall_m, precision_m, f1_score], optimizer='adam')
    return model

In [14]:
def build_gru_model(hp):
    # define hyperparameters parameters
    inteval_size = hp.Choice('interval_size', [3, 5, 7, 9, 11])
    num_layers = hp.Int('num_layers', 1, 7, step=2)
    activation = hp.Choice("activation", ["sigmoid", "relu", "tanh"])
    hiddent_units = hp.Choice('hiddent_units', [50, 100, 150, 200])
    n_features, n_outputs = 5, 2
    n_timesteps = 1

    # define model
    model = Sequential()
    # we want sigmoid in the first layer so that we convert the data between 0 and 1
    model.add(GRU(hiddent_units, activation='sigmoid', input_shape=(n_timesteps, n_features), return_sequences=num_layers > 1))
    for i in range(num_layers - 2):
        model.add(GRU(hiddent_units, activation=activation, return_sequences=True, dropout=hp.Float("dropout", 0.0 , 0.8 , 0.2)))
    if num_layers > 1:
        model.add(GRU(hiddent_units, activation=activation))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', metrics=['accuracy', recall_m, precision_m, f1_score], optimizer='adam')
    return model

# decision tree model
def build_model_decision_tree(hp):
    decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
    return decision_tree

In [38]:
train, test = split_dataset(df)
train_x, train_y = to_supervised(train, n_input)
val_x, test_y = to_supervised(test, n_input)
train_y = to_categorical(train_y, num_classes=2)
val_y = to_categorical(test_y, num_classes=2)
print(train_x, train_y)
print(val_x, val_y)

[[[23.914772 23.983334 23.96648  23.983147 23.743364]]

 [[23.983334 23.96648  23.983147 23.743364 23.542542]]

 [[23.96648  23.983147 23.743364 23.542542 23.686821]]

 [[23.983147 23.743364 23.542542 23.686821 23.40555 ]]

 [[23.743364 23.542542 23.686821 23.40555  23.707924]]

 [[23.542542 23.686821 23.40555  23.707924 23.755102]]

 [[23.686821 23.40555  23.707924 23.755102 23.267107]]

 [[23.40555  23.707924 23.755102 23.267107 23.65255 ]]

 [[23.707924 23.755102 23.267107 23.65255  23.603794]]

 [[23.755102 23.267107 23.65255  23.603794 23.68371 ]]

 [[23.267107 23.65255  23.603794 23.68371  23.418789]]

 [[23.65255  23.603794 23.68371  23.418789 23.743364]]

 [[23.603794 23.68371  23.418789 23.743364 23.542545]]

 [[23.68371  23.418789 23.743364 23.542545 23.686821]]

 [[23.418789 23.743364 23.542545 23.686821 23.40555 ]]

 [[23.743364 23.542545 23.686821 23.40555  23.707924]]

 [[23.542545 23.686821 23.40555  23.707924 23.755102]]

 [[23.686821 23.40555  23.707924 23.755102 23.26

In [39]:
class MyTuner(Hyperband):
    def  _build_and_fit_model(self, trial, fit_args, fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        if 'callbacks' in fit_kwargs:
            fit_kwargs['callbacks'] = [cb for cb in fit_kwargs['callbacks'] if not isinstance(cb, ModelCheckpoint)]
        return model.fit(*fit_args, **fit_kwargs)

In [40]:
tuner = MyTuner(build_gru_model,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='fault_classification_evaluation',
                     project_name='gru',
                     overwrite=True)

In [41]:
tuner.search(train_x, train_y,
             validation_split=0.2, callbacks=[])

Trial 33 Complete [00h 00m 16s]
val_accuracy: 0.5

Best val_accuracy So Far: 0.5
Total elapsed time: 00h 06m 03s

Search: Running Trial #34

Hyperparameter    |Value             |Best Value So Far 
interval_size     |3                 |3                 
num_layers        |3                 |3                 
activation        |tanh              |relu              
hiddent_units     |200               |100               
dropout           |0.2               |0                 
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |3                 |3                 
tuner/round       |0                 |0                 

Epoch 1/2


KeyboardInterrupt: 

In [18]:
# Print summary for same 
hyperparameter_results = defaultdict(list)
for file in glob.glob("fault_classification_evaluation/lstm/trial*/trial.json"):
    trial = json.load(open(file))
    if trial['status'] == 'COMPLETED' and trial['score'] is not None:
        hyperparameter_results[trial['trial_id'] + str(trial["hyperparameters"]["values"])].append(trial['metrics']['metrics']["val_f1_score"]['observations'][0]['value'][0])

for param, f1_scores in sorted(hyperparameter_results.items(), key=lambda a: sum(a[1]), reverse=True):
    mean_f1_score = sum(f1_scores)/len(f1_scores)
    max_f1_score = max(f1_scores)
    min_f1_score = min(f1_scores)
    variance_f1_score = (sum([(mean_f1_score-current_score)**2 for current_score in f1_scores]))/len(f1_scores)
    std_dev_f1_score = variance_f1_score ** (1/2)
    # summarize scores
    print("-----------------------------------------")
    print("hyperparameter tune",param)
    print("mean_f1_score",mean_f1_score)
    print("max_f1_score",max_f1_score)
    print("min_f1_score",min_f1_score)
    print("variance_f1_score",variance_f1_score)
    print("std_dev_f1_score",std_dev_f1_score)
    break

-----------------------------------------
hyperparameter tune 3f81c14fe6c5eb4380acd79454be6cd9{'num_layers': 5, 'activation': 'relu', 'hiddent_units': 200, 'dropout': 0.6000000000000001, 'tuner/epochs': 50, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
mean_f1_score 0.6666666269302368
max_f1_score 0.6666666269302368
min_f1_score 0.6666666269302368
variance_f1_score 0.0
std_dev_f1_score 0.0


In [19]:
## Kör det här!!!  
# load the new file
# split into train and test
algorithm_f1_score_list = []
algorithm_std_f1_score_list = []

algorithms = {
    'cnn-lstm': build_cnn_model,
    'lstm': build_lstm_model,
    'gru': build_gru_model
}

for algorithm, build_model in algorithms.items():
    print("--------------------------------------")
    print("algorithm",algorithm)
    train, test = split_dataset(df.values)
    # evaluate model and get scores
    # mean, maximum,minimum,std_dev,
    f1_scores = []
    for i in range(10):
        hp = kt.HyperParameters()
        model = build_model(hp)
        epochs = 10
        batch_size = 32

        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
        model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=False, callbacks=[callback])
        metrics = model.evaluate(val_x, val_y, return_dict=True, verbose=False)
        f1_scores.append(metrics['f1_score'])
    
    # Prepare and calculate results
    mean_f1_score = sum(f1_scores)/len(f1_scores)
    max_f1_score = max(f1_scores)
    min_f1_score = min(f1_scores)
    variance_f1_score = (sum([(mean_f1_score-current_score)**2 for current_score in f1_scores]))/len(f1_scores)
    std_dev_f1_score = variance_f1_score ** (1/2.0)
    # Add values to list
    algorithm_f1_score_list.append(mean_f1_score)
    algorithm_std_f1_score_list.append(std_dev_f1_score)
    # summarize scores
    print("mean_f1_score",mean_f1_score)
    print("max_f1_score",max_f1_score)
    print("min_f1_score",min_f1_score)
    print("variance_f1_score",variance_f1_score)
    print("std_dev_f1_score",std_dev_f1_score)
    # summarize_scores('cnn-lstm', mean_score, scores)
    break

--------------------------------------
algorithm cnn-lstm
mean_f1_score 0.6451612114906311
max_f1_score 0.6451612114906311
min_f1_score 0.6451612114906311
variance_f1_score 0.0
std_dev_f1_score 0.0
